In [ ]:
%load_ext autoreload
%autoreload 2

# IDQL 

> IDQL class
> 
>Title: Implicit Q-Learning with Diffusion Policy
>Author: Binjian Xin
>Date created: 2024/06/20
>Last modified: 2024/06/20
>Description: Agent from IDQL repo
>
>
>Title: IDQL: Implicit Q-learning as an Actor-Critic Meithod with Diffusion Policies
>Author: Philippe Hansen-Estruch, Ilya Kostrikov, Michael Janner, Jakub Grudzien Kuba, Sergey levine 
>Description: Implementing IDQL algorithm on VEOS.


In [ ]:
#| default_exp agent.idql

In [ ]:
#| export
import logging
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Tuple
import numpy as np
import pandas as pd
import tensorflow as tf
from typeguard import check_type

In [ ]:
#| export
from tspace.agent.utils.hyperparams import HyperParamRDPG
from tspace.storage.buffer.dask import DaskBuffer
from tspace.storage.buffer.mongo import MongoBuffer  # type: ignore
from tspace.data.core import PoolQuery  # type: ignore
from tspace.data.time import veos_lifetime_end_date, veos_lifetime_start_date

In [ ]:
#| export
from tspace.agent.dpg import DPG  # type: ignore
from tspace.agent.rdpg.seq_actor import SeqActor  # type: ignore
from tspace.agent.rdpg.seq_critic import SeqCritic  # type: ignore
from tspace.agent.utils.hyperparams import HyperParamRDPG  # type: ignore

In [ ]:

#| hide
from tspace.config.vehicles import Truck, TruckInCloud, trucks_by_id
from tspace.config.drivers import Driver
import logging
from typing import Union
from tspace.data.core import (
    RE_RECIPEKEY,
    ActionSpecs,
    ObservationMetaCloud,
    ObservationMetaECU,
    RewardSpecs,
    StateSpecsCloud,
    StateSpecsECU,
    get_filemeta_config,
)

from tspace.agent.utils.hyperparams import HyperParamDDPG  # type: ignore



In [ ]:
#| export
@dataclass
class IDQL(DPG):
    """IDQL agent for VEOS.

    Abstracts:

        data interface:
            - pool in mongodb
            - buffer in memory (numpy array)
        model interface:
            - actor network
            - critic network


    Attributes:

        actor_net: actor network
        critic_net: critic network
        target_actor_net: target actor network
        target_critic_net: target critic network
        _ckpt_actor_dir: checkpoint directory for actor
        _ckpt_critic_dir: checkpoint directory for critic
    """

    # Following are derived
    actor_net: Optional[SeqActor] = None  # actor_net_default
    critic_net: Optional[SeqCritic] = None  # critic_net_default
    target_actor_net: Optional[SeqActor] = None  # actor_net_default
    target_critic_net: Optional[SeqCritic] = None  # critic_net_default
    _ckpt_actor_dir: Optional[Path] = None  # Path("")
    _ckpt_critic_dir: Optional[Path] = None  # Path("")

    def __post_init__(
        self,
    ):
        """initialize the rdpg agent.

        args:
            truck.ObservationNumber (int): dimension of the state space.
            padding_value (float): value to pad the state with, impossible value for observation, action or re
        """

        self.logger = self.logger.getChild("eos").getChild(self.__str__())
        self.logger.propagate = True

        super().__post_init__()  # call DPG post_init for pool init and plot init
        self.coll_type = "EPISODE"
        self.hyper_param = HyperParamRDPG(
            HiddenDimension=256,
            PaddingValue=-10000,
            tbptt_k1=200,
            tbptt_k2=200,
            BatchSize=4,
            NStates=self.truck.observation_numel,
            NActions=self.truck.torque_flash_numel,
            ActionBias=self.truck.torque_bias,
            NLayerActor=2,
            NLayerCritic=2,
            Gamma=0.99,
            TauActor=0.005,
            TauCritic=0.005,
            ActorLR=0.001,
            CriticLR=0.001,
            CkptInterval=5,
        )

        self.buffer.query = PoolQuery(
            vehicle=self.truck.vid,
            driver=self.driver.pid,
            episodestart_start=veos_lifetime_start_date,
            episodestart_end=veos_lifetime_end_date,
            seq_len_from=1,  # from 10  # sample sequence with a length from 1 to 200
            seq_len_to=self.hyper_param.tbptt_k1 + 100,  # to 300
        )
        self.buffer.pool.query = self.buffer.query

        # actor network (w/ target network)
        self.init_checkpoint()

        self.actor_net = SeqActor(
            self.truck.observation_numel,
            self.truck.torque_flash_numel,
            self.hyper_param.HiddenDimension,  # 256
            self.hyper_param.NLayerActor,  # 2
            self.hyper_param.BatchSize,  # 4
            self.hyper_param.PaddingValue,  # -10000
            self.hyper_param.TauActor,  # 0.005
            self.hyper_param.ActorLR,  # 0.001
            self._ckpt_actor_dir,
            self.hyper_param.CkptInterval,  # 5
            self.logger,
            self.dict_logger,
        )

        self.target_actor_net = SeqActor(
            self.truck.observation_numel,
            self.truck.torque_flash_numel,
            self.hyper_param.HiddenDimension,  # 256
            self.hyper_param.NLayerActor,  # 2
            self.hyper_param.BatchSize,  # 4
            self.hyper_param.PaddingValue,  # -10000
            self.hyper_param.TauActor,  # 0.005
            self.hyper_param.ActorLR,  # 0.001
            self._ckpt_actor_dir,
            self.hyper_param.CkptInterval,  # 5
            self.logger,
            self.dict_logger,
        )
        # clone necessary for the first time training
        self.target_actor_net.clone_weights(self.actor_net)

        # critic network (w/ target network)

        self.critic_net = SeqCritic(
            self.truck.observation_numel,
            self.truck.torque_flash_numel,
            self.hyper_param.HiddenDimension,  # 256
            self.hyper_param.NLayerCritic,  # 2
            self.hyper_param.BatchSize,  # 4
            self.hyper_param.PaddingValue,  # -10000
            self.hyper_param.TauCritic,  # 0.005
            self.hyper_param.CriticLR,  # 0.002
            self._ckpt_critic_dir,
            self.hyper_param.CkptInterval,  # 5
            self.logger,
            self.dict_logger,
        )

        self.target_critic_net = SeqCritic(
            self.truck.observation_numel,
            self.truck.torque_flash_numel,
            self.hyper_param.HiddenDimension,  # 256
            self.hyper_param.NLayerCritic,  # 2
            self.hyper_param.BatchSize,  # 4
            self.hyper_param.PaddingValue,  # -10000
            self.hyper_param.TauCritic,  # 0.005
            self.hyper_param.CriticLR,  # 0.002
            self._ckpt_critic_dir,
            self.hyper_param.CkptInterval,  # 5
            self.logger,
            self.dict_logger,
        )
        # clone necessary for the first time training
        self.target_critic_net.clone_weights(self.critic_net)
        self.touch_gpu()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()